# MNIST Classification using PyTorch Lightning

In [19]:
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
import pytorch_lightning as pl
from torchmetrics.functional.classification.accuracy import accuracy

---
# PyTroch Lightning

1. model
2. optimizer 
3. data
4. training loop - The Magic
5. validation loop - The Validation Magic

In [30]:
# Defining ResNet as a Lightning Module
class ResNet(pl.LightningModule): # --> Model with residual connections
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(28 * 28, 64)
        self.l2 = nn.Linear(64, 64)
        self.l3 = nn.Linear(64, 10)
        self.do = nn.Dropout(0.1)

        self.loss = nn.CrossEntropyLoss()
    
    def forward(self, x):
        h1 = nn.functional.relu(self.l1(x))
        h2 = nn.functional.relu(self.l2(h1))
        do = self.do(h2 + h1)
        logits = self.l3(do)
        return logits

    def configure_optimizers(self):
        optimizer = optim.SGD(self.parameters(), lr=1e-2)
        return optimizer

    def training_step(self, batch, batch_idx):
        x, y = batch

        # x: b * 1 * 28 * 28
        b = x.size(0)
        x = x.view(b, -1)

        # 1: Forward Prop
        logits = self(x)

        # 2: Compute Objective / Loss Function
        J = self.loss(logits, y)

        acc = accuracy(logits, y)
        self.log("acc", acc, prog_bar=True)
        return {'loss': J}
    
    def train_dataloader(self):
        # Train, val split
        train_data = datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor())
        # train, val = random_split(train_data, [55000, 5000])
        train_loader = DataLoader(train_data, batch_size=64, num_workers=8)
        # val_loader = DataLoader(val, batch_size=32)
        return train_loader

model = ResNet()

In [31]:
trainer = pl.Trainer(max_epochs=5, gpus=1)
trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type             | Params
------------------------------------------
0 | l1   | Linear           | 50.2 K
1 | l2   | Linear           | 4.2 K 
2 | l3   | Linear           | 650   
3 | do   | Dropout          | 0     
4 | loss | CrossEntropyLoss | 0     
------------------------------------------
55.1 K    Trainable params
0         Non-trainable params
55.1 K    Total params
0.220     Total estimated model params size (MB)


Epoch 4:  90%|█████████ | 845/938 [00:28<00:03, 29.58it/s, loss=0.329, v_num=8, acc=0.891]

In [3]:
class ImageClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = ResNet()